In [1]:
from bs4 import BeautifulSoup
import requests
from unidecode import unidecode

In [76]:
url = 'https://archiveofourown.org/tags/%E0%B9%80%E0%B8%94%E0%B8%B7%E0%B8%AD%E0%B8%99%E0%B9%80%E0%B8%81%E0%B8%B5%E0%B9%89%E0%B8%A2%E0%B8%A7%E0%B9%80%E0%B8%94%E0%B8%B7%E0%B8%AD%E0%B8%99%20%7C%202%20Moons%20-%20Chiffon_Cake/works'
url = 'https://archiveofourown.org/tags/221B%20-%20Vincent%20Starrett/works'
url = 'https://archiveofourown.org/tags/Star%20Wars:%20The%20Clone%20Wars%20(2008)%20-%20All%20Media%20Types/works'


In [77]:
#headers = {'user-agent' : header_info}
req = requests.get(url)
src = req.text
soup = BeautifulSoup(src, 'html.parser')


In [122]:
def get_tags(meta):
    '''
    returns a list of lists, of
    rating, category, fandom, pairing, characters, additional_tags
    '''
    tags = ['relationships', 'characters', 'freeforms']
    return list(map(lambda tag: get_tag_info(tag, meta), tags))
    

In [111]:
def get_tag_info(category, meta):
    '''
    given a category and a 'work meta group, returns a list of tags (eg, 'rating' -> 'explicit')
    '''
    try:
        tag_list = meta.find_all("li", class_=str(category))
    except AttributeError as e:
        return []
    return [unidecode(result.text) for result in tag_list] 

In [112]:
def get_required_tags(work):
    req_tags = work.find(class_='required-tags').find_all('a')
    return [x.text for x in req_tags]

In [113]:
def get_stats(work):
    '''
    returns a list of  
    language, published, status, date status, words, chapters, comments, kudos, bookmarks, hits
    '''
    categories = ['language', 'words', 'chapters', 'collections', 'comments', 'kudos', 'bookmarks', 'hits'] 
    stats = []
    for cat in categories:
        try:
            result = work.find("dd", class_=cat).text
        except:
            result = ''
        stats.append(result)
    return stats
  

In [114]:
def get_header(work):
    '''
    returns work_id, title, author, gifted to user
    '''
    
    result = work.find('h4', class_='heading').find_all('a')
    work_id = result[0].get('href').strip('/works/')
    title = result[0].text
    author = result[1].text
    if len(result) == 3:
        gifted = result[2].text
    else:
        gifted = ''
    return [work_id, title, author, gifted]

In [115]:
def get_fandoms(work):
    try:
        tag_list = work.find('h5', class_='fandoms heading').find_all('a')
    except AttributeError as e:
        return []
    return [unidecode(result.text) for result in tag_list]    

In [125]:
def get_summary(work):
    try:
        summary = work.find('blockquote', class_='userstuff summary').text.strip()
    except AttributeError as e:
        summary = ''
    return [summary]

In [117]:
def get_updated(work):
    try:
        date = work.find('p', class_='datetime').text
    except AttributeError as e:
        date = ''
    return [date]

In [131]:
import datetime
x = datetime.datetime.now()
print(x.strftime("%b%d%Y"))
scrape_date = datetime.datetime.now().strftime("%d %b %Y")

Mar052020


In [119]:
def get_series(work):
    try:
        series = work.find('ul', class_='series')
        part = series.find('strong').text
        s_name = series.find('a').text
    except AttributeError as e:
        part, s_name = '', ''
    return [part, s_name]

In [133]:
works = soup.find_all(class_="work blurb group")
for work in works:
    tags = get_tags(work)
    req_tags = get_required_tags(work)
    stats = get_stats(work)
    header_tags = get_header(work)
    fandoms = get_fandoms(work)
    summary = get_summary(work)
    updated = get_updated(work)
    series = get_series(work)
    header = ['work_id', 'title', 'author', 'gifted', 'rating', 'warnings', 'category', 'status', 'fandom', 'relationship', 'character', 'additional tags', 'summary','language', 'words', 'chapters', 'collections', 'comments', 'kudos', 'bookmarks', 'hits', 'series_part', 'series_name', 'updated', 'scrape_date']
    row = header_tags + req_tags + fandoms + list(map(lambda x: ', '.join(x), tags)) + summary + stats + series + updated + [scrape_date]
    row = header_tags + req_tags + fandoms + list(map(lambda x: ', '.join(x), tags)) + summary + stats + series + updated + [scrape_date]
    print(header)
    print(f"Row: {row}")
    print("*****") 

['work_id', 'title', 'author', 'gifted', 'rating', 'warnings', 'category', 'status', 'fandom', 'relationship', 'character', 'additional tags', 'summary', 'language', 'words', 'chapters', 'collections', 'comments', 'kudos', 'bookmarks', 'hits', 'series_part', 'series_name', 'updated', 'scrape_date']
Row: ['23032642', 'A Master’s Hold', 'Paranormal_Shitness', '', 'Mature', 'No Archive Warnings Apply', 'F/M, M/M, Multi', 'Complete Work', 'Star Wars Prequel Trilogy', 'Star Wars: The Clone Wars (2008) - All Media Types', 'Obi-Wan Kenobi/Anakin Skywalker, Padme Amidala/Anakin Skywalker, Obi-Wan Kenobi/Satine Kryze', 'Anakin Skywalker, Obi-Wan Kenobi, Padme Amidala', "Oral Sex, Force Bond, Master/Padawan, Anakin Won't Take No For An Answer, Which is Alright Because Obi-Wan Doesn't Really Want To Say It, dubcon, Anakin Is Still Too Horny, Padme Is Unflappable, Obi-Wan Has Never Gotten A Blowjob In His Life, Sexually Repressed Jedi", 'Obi-Wan struggles to maintain control of his pupil even as A

['work_id', 'title', 'author', 'gifted', 'rating', 'warnings', 'category', 'status', 'fandom', 'relationship', 'character', 'additional tags', 'summary', 'language', 'words', 'chapters', 'collections', 'comments', 'kudos', 'bookmarks', 'hits', 'series_part', 'series_name', 'updated', 'scrape_date']
Row: ['23028571', 'Maybe', 'MinnieTheMoocherDA', '', 'General Audiences', 'No Archive Warnings Apply', 'M/M', 'Complete Work', 'Star Wars: The Clone Wars (2008) - All Media Types', 'Star Wars - All Media Types', 'Star Wars: Rebels', 'Gobi Glie/Cham Syndulla, Gobi Glie & Cham Syndulla, Kanan Jarrus/Hera Syndulla, Cham Syndulla & Hera Syndulla, CC-6454 | Ponds/Mace Windu', 'Gobi Glie, Cham Syndulla, Hera Syndulla, Mace Windu', '', 'Gobi tries to talk Cham into letting him go on the mission to free the Capital.', 'English', '554', '1/1', '', '', '3', '1', '13', '8', 'Clone wars drabbles', '05 Mar 2020', '05 Mar 2020']
*****


In [108]:
tag1 = ['stuff1', 'morestuff1']
tag2 = ['']
tag3 = [['boop3'],['blep3']]
collated = [tag1]+[tag2]+tag3
print(collated)

[['stuff1', 'morestuff1'], [''], ['boop3'], ['blep3']]


In [ ]:
#url = 'http://archiveofourown.org/works/'+str(22336816)+'/kudos'  
url = 'http://archiveofourown.org/works/'+str(14737352)+'/kudos' 
#url = 'http://archiveofourown.org/works/'+str(22336817)+'/kudos' # 404
headers = {'user-agent' : 'Homemade Bot; rebecca.sanjabi@gmail.com'}
    
req = requests.get(url, headers=headers)
src = req.text
soup = BeautifulSoup(src, 'html.parser')
print(url)

In [ ]:
def get_kudos(soup):
    try:
        my_list = soup.find(id="kudos").find_all('a')
        secondary_list = [x.text for x in set(my_list)]
        print("try")
    except AttributeError as e:
        print("except")
        return []
    else:
        print("else")
        return secondary_list

In [ ]:
get_kudos(soup)

In [ ]:
if req.status_code != 200:
    print('Access Denied')
my_kudo_list = get_kudos(soup)
unique_list = sorted(my_kudo_list)
for el in unique_list:
    print(el)
print(len(unique_list))

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/swr_fanworks/fanworks_kudos2.csv')

In [ ]:
df.head()

In [ ]:
df['user'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(1, figsize=(9, 6))

# Create an axes instance
ax = fig.add_subplot(111)

#ax.set_xticklabels(['English', 'Polski','Russkii'])
#ax.set_xlabel(metric + ' and Languages')
ax.set_ylim([0, 100])

# Create the boxplot
bp = ax.boxplot(df['user'].value_counts())

# Save the figure
fig.savefig('fig1.png', bbox_inches='tight')

plt.show()

In [ ]:
df.loc[df['user']=='pepoluan']

In [ ]:
df.info()

In [ ]:
len(df['work_id'].unique())

In [ ]:
len(df['user'].unique())

In [ ]:
columns_titles = ["work_id","user"]
df2=df.reindex(columns=columns_titles)

In [ ]:
df2.head()

In [ ]:
df2.to_csv('../data/swr_fanworks/fanworks_kudos2.csv', index=False)

In [ ]:
def my_func(var, var2):
    var = var + 3
    var2 = var2/4
    print(f"Inside function var value: {var}")
    
    return var

In [ ]:
var = 1
var2 = 4
print(f"Prior to function var value: {var}")
my_func(var, var2)
print(f"Post function var value: {var}")


In [ ]:
def reassign(list):
  list = [0, 1]

def append(list):
  list.append(1)

list = [0]
print(list)
reassign(list)
print(list)
append(list)
print(list)

In [ ]:
listA = [0]
listB = listA
listB.append(1)
print(listA)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer

%matplotlib inline

plt.style.use('bmh')
pd.options.display.float_format = '{:,.0f}'.format
pd.set_option("display.max_columns", 40)

In [ ]:
df = pd.read_csv('../data/fanfics_metadata.csv')
cols_with_missing = [col for col in df.columns if df[col].isnull().any()]
cols_with_missing

In [ ]:
df['category'].fillna('No Category Specified', inplace=True)
df['relationship'].fillna('No Relationship Specified', inplace=True)
df['character'].fillna('No Character Specified', inplace=True)
df['additional tags'].fillna('No Additional Tags Specified', inplace=True)
df['words'].fillna(0, inplace=True)
df['comments'].fillna(0, inplace=True)
df['kudos'].fillna(0, inplace=True)
df['bookmarks'].fillna(0, inplace=True)
df['hits'].fillna(0, inplace=True)

In [ ]:
df['kudos'].describe()

In [ ]:
df['kudos'].value_counts().sort_values()

In [ ]:
df.loc[df['kudos'] != 0].count()

In [ ]:
from scipy.sparse import csr_matrix

docs = [["hello", "world", "hello"], ["goodbye", "cruel", "world"]]
indptr = [0]
indices = []
data = []
vocabulary = {}
for d in docs:
    for term in d:
        index = vocabulary.setdefault(term, len(vocabulary))
        indices.append(index)
        data.append(1)
        indptr.append(len(indices))
        print(f"term: {term} index: {index} indices: {indices} data: {data} vocab: {vocabulary}")
csr_matrix((data, indices, indptr), dtype=int).toarray()


In [ ]:
str = '255.255.255.255'
str.replace('.', '[.]')